# Лабораторная работа 1.1 
Трунов Антон М8О-304Б Вариант 16

# Задание
Реализовать алгоритм LU -  разложения матриц (с выбором главного элемента) в виде программы. Используя разработанное программное обеспечение, решить систему линейных алгебраических уравнений (СЛАУ). Для матрицы СЛАУ вычислить определитель и обратную матрицу. 
\begin{cases}
-5x_1 - x_2 - 3x_3 - x_4 = 18 \\
-2x_1 + 8x_3 - 4x_4 = -12 \\
-7x_1 - 2x_2 + 2x_3 - 2x_4 = 6 \\
2x_1 - 4x_2 - 4x_3 + 4x_4 = -12 \\
\end{cases}

# $LUP$ разложение квадратной матрицы

In [3]:
import numpy as np

def PLU(a):
    N = a.shape[0]
    u = a.copy()
    L = np.eye(N)
    count = 0
    Parr = np.arange(N) #permutation array
    for j in range(N - 1):
        maxindex =  np.where(abs(u[j:, j]) == np.amax(abs(u[j:, j])))[0][0]#searching maximum index in column
        if j != j+maxindex:
            count+=1
        Parr[j], Parr[maxindex+j] =Parr[maxindex+j], Parr[j] #adding permutations into array
        u[[j,j+maxindex], :] = u[[j+maxindex,j], :]
        gamma = u[j + 1:, j] / u[j, j]
        for i in range(N-1-j):
            u[i+j+1,:]=-gamma[i]*u[j,:]+u[i+1+j,:]
        L[j+1:,j]=gamma[:]
        L[[j,j+maxindex], :j] = L[[j+maxindex,j], :j]
    detA = det(u)
    return L, u, Parr, (detA * (-1)**count)

Вычисление определителя.

In [4]:
def det(U):
    detA = 1
    for i in range(U.shape[0]):
        detA *= U[i,i]
    return detA

Решение СЛАУ.
$$
Ax=b \\
PA = Pb \\
LUx = b'\\
Ly = b' \\
Ux = y \\
$$

In [5]:
def solveSLAU(Parr,L,U,b):#LUx=Pb
    N = len(b)
    Pb = np.copy(b)
    y=np.zeros(N)
    x=np.zeros(N)
    for i in range(N): #P*b
        Pb[i] = b[Parr[i]]
    for i in range(N):#Ly=Pb
        y[i] = Pb[i]-np.inner(y[:i],L[i,:i])
    for i in range(N-1,-1,-1):#Ux=y
        x[i] = (y[i]-np.inner(x[i+1:],U[i,i+1:]))/U[i,i]
    return x

Обратная матрица
$$
PA = B \\
BB^{-1} = E \\
LUB^{-1} = E\\
LY = E \\
UB^{-1} = Y \\
$$
Так же можем установить соотношение между отбратной $A$ и  $B$
$$
A^{-1}P^{-1}=B^{-1} \\
A^{-1}=B^{-1}P
$$


In [6]:
def inverse(Parr, L, U):
    N= len(b)#let PA = B = LU and B_-1=X then BX = E : LUX = E where LY = E, UX = Y therefore A_-1 = XP
    E = np.eye(N)
    Y = np.zeros((N,N))
    X = np.zeros((N,N))
    P= np.zeros((N,N))
    for j in range(N):
        for i in range(N):
            Y[i,j]= E[i,j] - np.inner(Y[:i,j],L[i,:i])
    for j in range(N):
        for i in range(N-1,-1,-1):
            X[i, j] = (Y[i, j] - np.inner(X[i+1:,j], U[i,i+1:]))/U[i,i]
    for i in range(N):
        P[i, Parr[i]] = 1
    X = X @ P
    return X

In [7]:
np.set_printoptions(precision=3)
a = np.array([[-5,-1,-3,-1],[-2,0,8,-4],[-7,-2,2,-2],[2,-4,-4,4]],dtype=float)
b = np.array([18,-12,6,-12],dtype=float)

In [8]:
L, u, Parr, detA = PLU(a)
x = solveSLAU(Parr,L,u,b)
invA = inverse(Parr,L,u)

In [9]:
print("Determinant:",detA,"\n")
print("Solution:",x,"\n")
print("Inverse A:\n",invA,"\n")
print("Using np.linalg\n")
print("Determinant:",np.linalg.det(a),"\n")
print("Solution:",np.linalg.solve(a,b),"\n")
print("Inverse A:\n",np.linalg.inv(a),"\n")

Determinant: 288.0 

Solution: [-2.  3. -3. -2.] 

Inverse A:
 [[ 0.222  0.222 -0.333  0.111]
 [-0.333 -0.333  0.25  -0.292]
 [-0.333 -0.083  0.25  -0.042]
 [-0.778 -0.528  0.667 -0.139]] 

Using np.linalg

Determinant: 288.00000000000006 

Solution: [-2.  3. -3. -2.] 

Inverse A:
 [[ 0.222  0.222 -0.333  0.111]
 [-0.333 -0.333  0.25  -0.292]
 [-0.333 -0.083  0.25  -0.042]
 [-0.778 -0.528  0.667 -0.139]] 



In [10]:
def LUprint(L,u,Parr,a):
    N = a.shape[0]
    P = np.zeros((N,N)) #making inverse permutation matrix from Parr
    for i in range(N):
        P[Parr[i],i]=1
    print("A:\n",a,"\n")
    print("L:\n",L, "\n") #Permutated L
    print("U:\n",u, "\n") #Permutated U
    print("P:\n",P,"\n")#Its result is a permutated matrix a
    print(L@u)

LUprint(L,u, Parr,a)

A:
 [[-5. -1. -3. -1.]
 [-2.  0.  8. -4.]
 [-7. -2.  2. -2.]
 [ 2. -4. -4.  4.]] 

L:
 [[ 1.     0.     0.     0.   ]
 [-0.286  1.     0.     0.   ]
 [ 0.286 -0.125  1.     0.   ]
 [ 0.714 -0.094 -0.679  1.   ]] 

U:
 [[-7.    -2.     2.    -2.   ]
 [ 0.    -4.571 -3.429  3.429]
 [ 0.     0.     7.    -3.   ]
 [ 0.     0.     0.    -1.286]] 

P:
 [[0. 0. 0. 1.]
 [0. 0. 1. 0.]
 [1. 0. 0. 0.]
 [0. 1. 0. 0.]] 

[[-7. -2.  2. -2.]
 [ 2. -4. -4.  4.]
 [-2.  0.  8. -4.]
 [-5. -1. -3. -1.]]
